# Build a Crew to Tailor Job Applications

The libraries are already installed. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

**Note**: 
- You can use `gpt-4-turbo` when you run the notebook _locally_ 

In [ ]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = "sk-************"
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = "SERPER_API_KEY"


## crewAI Tools

In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [ ]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output from those tasks.

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [ ]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

2025-07-13 20:33:32,712 - 139938651577216 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input.

In [ ]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

Thought: Since accessing the job posting directly and attempting to find a cached version have failed, I will explore an alternative approach. I will delegate the task to the 'Personal Profiler for Engineers' coworker to find a similar job posting by AI Fund, which could likely have similar requirements as the one that was originally provided.

Action: Delegate work to co-worker
Action Input: {"coworker": "Personal Profiler for Engineers", "task": "Find similar job posting", "context": "The original job posting at https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1 was inaccessible due to a 404 error. The task is to find a similar job posting by AI Fund that can provide insights into the typical job requirements for similar positions at the company."}

> Entering new CrewAgentExecutor chain...
Thought: Since the original job posting is inaccessible, I need to find similar job postings by AI Fund to understand the typical job requirements for similar positions. I will star

Thought: I now can give a great answer based on the detailed job listings I found on AI Fund's main job listing page.

Final Answer: Here are some similar job postings by AI Fund that might give insights into the typical job requirements for similar positions at the company:

1. **AI Engineer**
   - **Type**: Hybrid — Full-time
   - **Location**: Palo Alto, CA
   - **Application Link**: [Apply Here](https://jobs.lever.co/AIFund)

2. **AI Fund-Principal**
   - **Type**: Hybrid — Full-time
   - **Location**: Palo Alto, CA
   - **Application Link**: [Apply Here](https://jobs.lever.co/AIFund)

3. **Founder-in-Residence (CapSure)**
   - **Type**: Remote — Full-time
   - **Location**: Remote
   - **Application Link**: [Apply Here](https://jobs.lever.co/AIFund)

4. **Technical Builder [Senior AI Full-Stack Engineer]**
   - **Type**: On-site — Full-time
   - **Location**: Palo Alto, CA
   - **Application Link**: [Apply Here](https://jobs.lever.co/AIFund)

5. **Full-stack Engineer (Generative A

Thought: To create a resume that resonates well with the job requirements at AI Fund, I will focus on emphasizing the most relevant skills and experiences from Noah's profile. I will adjust each section of his existing resume to better align with the roles he might be interested in, such as AI Engineer, AI Fund-Principal, Founder-in-Residence, Senior AI Full-Stack Engineer, and Full-stack Engineer for Generative AI Prototyping & MVP.

Final Answer:
# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished leader in software engineering with over 18 years in the technology sector. Specializing in AI solutions and data science, he has successfully led both remote and in-office teams across multiple high-stakes projects. Noah holds an MBA and is noted for his strategic insights and adeptness in programming languages including Python, Ruby, and Elixir. His strong foundation in AI has positioned him as a key player in deploying

Thought: 
With Noah's detailed professional profile, I can now create tailored interview questions and talking points that align with the roles at AI Fund. These questions will cover his technical skills, leadership experiences, and specific projects he has worked on, ensuring they are relevant to the roles he might be applying for, such as AI Engineer, AI Fund-Principal, and other senior AI-related positions.

Final Answer:
**Interview Questions and Talking Points for Noah Williams**

1. **For the Role of AI Engineer:**
   - **Question:** "Can you discuss a specific instance where you utilized TensorFlow or PyTorch in a project at DataKernel, and how did it impact the project's outcome?"
   - **Talking Point:** Highlight your role in integrating cutting-edge AI technologies at DataKernel and how you enhanced the product capabilities, aligning with AI Engineer skills requirements.

2. **For the Role of AI Fund-Principal:**
   - **Question:** "With your leadership experience at DataKern

- Display the generated `tailored_resume.md` file.

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished leader in software engineering with over 18 years in the technology sector. Specializing in AI solutions and data science, he has successfully led both remote and in-office teams across multiple high-stakes projects. Noah holds an MBA and is noted for his strategic insights and adeptness in programming languages including Python, Ruby, and Elixir. His strong foundation in AI has positioned him as a key player in deploying scalable AI technologies and innovative data models.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division into a key revenue pillar by integrating cutting-edge AI technologies and scalable vector databases.
- Directed strategic AI initiatives that enhanced product capabilities and market positioning, aligning with industry benchmarks.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Oversaw engineering strategies in collaboration with C-level executives, enhancing the company's technological trajectory with a focus on AI-driven projects.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Founded and led a startup focused on innovative IoT solutions for pet care, leveraging AI technologies for product development and market strategies.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Managed and mentored multiple engineering teams, emphasizing innovation and the integration of AI technologies in product development.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Spearheaded the development of data-driven tools and AI applications that revolutionized customer behavior analysis and business decision-making processes.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Played a crucial role in the development and optimization of AI-driven APIs, enhancing system performance and user engagement.

### BetCraft: CTO — 2013 - 2015
- Guided technological strategy and development post-Series A funding, focusing on AI and advanced data analytics platforms.

## Technical Skills
- **Languages**: Python, Ruby, Elixir
- **Frameworks/Technologies**: Rails, Phoenix, TensorFlow, PyTorch
- **Specializations**: AI Model Development, Machine Learning, Data Science, Full Stack Development

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

With his extensive background in AI and software engineering, coupled with strategic leadership in various high-profile tech projects, Noah is ideally positioned for roles that require deep technical expertise and visionary leadership in AI technologies.

- Display the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for Noah Williams**

1. **For the Role of AI Engineer:**
   - **Question:** "Can you discuss a specific instance where you utilized TensorFlow or PyTorch in a project at DataKernel, and how did it impact the project's outcome?"
   - **Talking Point:** Highlight your role in integrating cutting-edge AI technologies at DataKernel and how you enhanced the product capabilities, aligning with AI Engineer skills requirements.

2. **For the Role of AI Fund-Principal:**
   - **Question:** "With your leadership experience at DataKernel and InnovPet, how do you approach strategic planning and execution in AI projects?"
   - **Talking Point:** Discuss your strategic initiatives and the adoption of AI technologies that set new benchmarks in the industry, demonstrating your qualifications for a leadership role in AI.

3. **For the Role of Founder-in-Residence (CapSure):**
   - **Question:** "As the founder of InnovPet, what were the biggest challenges you faced in the startup ecosystem, and how did you overcome them?"
   - **Talking Point:** Elaborate on your entrepreneurial skills, experience in securing funding, and navigating initial product launches, relevant to the Founder-in-Residence role.

4. **For the Role of Technical Builder [Senior AI Full-Stack Engineer]:**
   - **Question:** "Can you provide an example of a full-stack project where you successfully integrated AI technologies, particularly during your time managing engineering teams at EliteDevs?"
   - **Talking Point:** Focus on your proficiency in both front-end and back-end development and how you incorporated AI into these projects, matching the Technical Builder role's skillset.

5. **For the Role of Full-stack Engineer (Generative AI Prototyping & MVP):**
   - **Question:** "In your experience, what is critical when developing MVPs for AI applications, and could you share an example from your projects?"
   - **Talking Point:** Discuss your hands-on experience with rapid prototyping and AI-driven product design, particularly highlighting any generative AI technologies used in your past roles.

**General Questions:**
   - **Question:** "How do you ensure effective communication and collaboration within your remote teams across different time zones?"
   - **Talking Point:** Talk about your leadership style and methods for fostering a culture of transparency and mutual respect, which is crucial for the remote roles at AI Fund.

**Preparation Tips:**
   - Review specific AI technologies and projects mentioned in your resume to provide detailed examples during the interview.
   - Be prepared to discuss how your strategic leadership directly influenced business outcomes and technological advancements.
   - Reflect on your past roles and think about how they align with the requirements and expectations of the positions you are applying for at AI Fund.

These questions and talking points are designed to help Noah effectively showcase his expertise, experience, and fit for the various roles at AI Fund during the interview process.